# 📍 Biomechanical Analysis - Joint Angles & Load Estimation

This notebook calculates joint angles (elbow, knee, hip) and estimates the biomechanical load on key body parts (arms, legs, hips, chest) using 2D pose keypoints extracted via YOLOv8.

**Assumptions**:
- Body weight: 60 kg
- Dumbbell weight: 5.5 kg in each hand
- Static poses only


# Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import math
import os

# 1. Joint angle calculations (elbow, knee, hip)

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Define Google Drive save path
drive_folder = "/content/drive/MyDrive/SEE_Assessment"
os.makedirs(drive_folder, exist_ok=True)

 Load the 2D Keypoints CSV

In [ ]:
# Load keypoints downloaded and saved from the pose estimation notebook
df_keypoints = pd.read_csv('/content/drive/MyDrive/SEE_Assessment/keypoints.csv')

# Preview keypoints
print("Loaded Keypoints DataFrame:")
df_keypoints.head()

Loaded Keypoints DataFrame:


,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,aaron-brogden-9y4MaTz2Js0-unsplash.jpg,1401.332642,1597.854248,1446.944336,1525.367065,1358.923218,1488.737671,0.000000,0.000000,1135.094727,...,1019.826477,3109.603271,1205.262939,3840.495361,1034.875000,3934.542969,0.000000,0.000000,0.000000,0.000000
1,pexels-shvetsa-4587381 (2).jpg,2427.232178,996.484314,2471.211182,882.327026,2359.252197,871.895264,0.000000,0.000000,2057.656738,...,1895.375488,2790.837646,3195.959717,2483.524170,1931.501587,3489.845703,2956.810303,3543.261475,0.000000,0.000000
2,aaron-brogden-9y4MaTz2Js0-unsplash (1).jpg,1401.332642,1597.854248,1446.944336,1525.367065,1358.923218,1488.737671,0.000000,0.000000,1135.094727,...,1019.826477,3109.603271,1205.262939,3840.495361,1034.875000,3934.542969,0.000000,0.000000,0.000000,0.000000
3,anastase-maragos-iUzgePOoGko-unsplash (1).jpg,1859.202026,1812.019775,1921.308350,1760.410156,1823.065430,1765.839111,2068.910645,1827.673096,0.000000,...,1785.220215,3034.496582,2194.188965,3755.903076,1790.057617,3750.530029,2263.456055,4505.608887,1792.108276,4490.726562
4,john-fornander-TAZoUmDqzXk-unsplash.jpg,2121.877930,3358.743652,2171.137939,3230.602295,2080.133301,3278.068848,2452.163330,3098.942627,0.000000,...,2376.989746,5200.759277,2894.450195,6393.967285,2422.261963,6286.392090,3463.841553,7189.644043,2639.279541,7349.532715


Define Utility Function to Calculate Joint Angles

In [ ]:
def calculate_angle(a, b, c):
    """
    Calculates the angle at point b (in degrees), given 3 points: a, b, c
    a, b, c: (x, y) coordinate tuples
    """
    a, b, c = np.array(a), np.array(b), np.array(c)

    # Vectors: BA and BC
    ba = a - b
    bc = c - b

    # Calculate cosine of angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))

    # Prevent numerical errors/ Handle possible float precision issues
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)

    # Compute angle in degrees
    angle = np.degrees(np.arccos(cosine_angle))
    return round(angle, 2)


Loop Over the Keypoints DataFrame and Calculate Angles

*  Left & Right Elbow Angle

*   Left & Right Knee Angle

*   Left & Right Hip Angle







In [ ]:
# List to store angle results
angle_records = []

for _, row in df_keypoints.iterrows():
    img_name = row['image']

    # Helper function to get a joint's (x, y) coordinates
    def pt(joint): return (row[f"{joint}_x"], row[f"{joint}_y"])

    try:
        # Elbow: shoulder–elbow–wrist
        left_elbow_angle = calculate_angle(pt('left_shoulder'), pt('left_elbow'), pt('left_wrist'))
        right_elbow_angle = calculate_angle(pt('right_shoulder'), pt('right_elbow'), pt('right_wrist'))

        # Knee: hip–knee–ankle
        left_knee_angle = calculate_angle(pt('left_hip'), pt('left_knee'), pt('left_ankle'))
        right_knee_angle = calculate_angle(pt('right_hip'), pt('right_knee'), pt('right_ankle'))

        # Hip: shoulder–hip–knee
        left_hip_angle = calculate_angle(pt('left_shoulder'), pt('left_hip'), pt('left_knee'))
        right_hip_angle = calculate_angle(pt('right_shoulder'), pt('right_hip'), pt('right_knee'))

        angle_records.append({
            'image': img_name,
            'left_elbow_angle': left_elbow_angle,
            'right_elbow_angle': right_elbow_angle,
            'left_knee_angle': left_knee_angle,
            'right_knee_angle': right_knee_angle,
            'left_hip_angle': left_hip_angle,
            'right_hip_angle': right_hip_angle
        })

    except Exception as e:
        print(f"Error processing {img_name}: {e}")


<ipython-input-15-63f781ed648a>:13: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))


***Store*** Angles to DataFrame & ***Save*** Joint Angles

In [ ]:
# -------------------------------
# ✅ Save Joint Angles
# -------------------------------
df_angles = pd.DataFrame(angle_records)
print("📐 Joint Angle Calculations (Degrees):")
display(df_angles.head())

# Save to /content/
df_angles.to_csv("/content/joint_angles.csv", index=False)
print("✅ Joint angles saved to /content/joint_angles.csv")

# Save to Google Drive
drive_angles_path = os.path.join(drive_folder, "joint_angles.csv")
df_angles.to_csv(drive_angles_path, index=False)
print(f"✅ Joint angles also saved to {drive_angles_path}")

📐 Joint Angle Calculations (Degrees):


,image,left_elbow_angle,right_elbow_angle,left_knee_angle,right_knee_angle,left_hip_angle,right_hip_angle
0,aaron-brogden-9y4MaTz2Js0-unsplash.jpg,171.02,161.28,23.05,13.69,173.24,173.66
1,pexels-shvetsa-4587381 (2).jpg,61.18,173.69,72.37,26.00,90.89,171.85
2,aaron-brogden-9y4MaTz2Js0-unsplash (1).jpg,171.02,161.28,23.05,13.69,173.24,173.66
3,anastase-maragos-iUzgePOoGko-unsplash (1).jpg,164.96,177.25,179.94,179.77,166.39,176.94
4,john-fornander-TAZoUmDqzXk-unsplash.jpg,145.93,168.15,149.36,170.85,169.48,179.10


✅ Joint angles saved to /content/joint_angles.csv
✅ Joint angles also saved to /content/drive/MyDrive/SEE_Assessment/joint_angles.csv


# 2. Biomechanical load estimation (arms, legs, hips, chest)

Define Load Estimation Function

In [ ]:
def estimate_load_distribution(row, body_weight=60, dumbbell_weight=5.5):
    """
    Estimate biomechanical load on body parts based on joint angles.
    Output loads are in kg.
    """
    # Extract angles
    le, re = row['left_elbow_angle'], row['right_elbow_angle']
    lk, rk = row['left_knee_angle'], row['right_knee_angle']
    lh, rh = row['left_hip_angle'], row['right_hip_angle']

    # ------------------ Arm Load ------------------
    def arm_load(angle):
        return dumbbell_weight * (1 + (180 - angle) / 90)  # more extension = more stress

    left_arm = arm_load(le)
    right_arm = arm_load(re)

    # ------------------ Leg Load ------------------
    avg_knee_angle = (lk + rk) / 2
    leg_factor = (180 - avg_knee_angle) / 90  # squatting = more load
    leg_load = body_weight * 0.5 * (1 + leg_factor)

    # ------------------ Hip Load ------------------
    avg_hip_angle = (lh + rh) / 2
    hip_load = body_weight * 0.3 * (1 + (180 - avg_hip_angle) / 120)

    # ------------------ Chest Load ------------------
    chest_load = body_weight * 0.2 * (1 + (180 - avg_hip_angle) / 90)

    return {
        'image': row['image'],
        'left_arm_load_kg': round(left_arm, 2),
        'right_arm_load_kg': round(right_arm, 2),
        'leg_load_kg': round(leg_load, 2),
        'hip_load_kg': round(hip_load, 2),
        'chest_load_kg': round(chest_load, 2)
    }


Apply Load Estimation to All Images

In [ ]:
# -------------------------------
# ✅ Estimate and Save Loads
# -------------------------------
# Apply biomechanical estimation row by row
load_estimations = [estimate_load_distribution(row) for _, row in df_angles.iterrows()]
df_loads = pd.DataFrame(load_estimations)

print("🏋️ Biomechanical Load Estimations:")
display(df_loads.head())

# Save to /content/
df_loads.to_csv("/content/load_estimations.csv", index=False)
print("✅ Load estimations saved to /content/load_estimations.csv")

# Save to Google Drive
drive_loads_path = os.path.join(drive_folder, "load_estimations.csv")
df_loads.to_csv(drive_loads_path, index=False)
print(f"✅ Load estimations also saved to {drive_loads_path}")


🏋️ Biomechanical Load Estimations:


,image,left_arm_load_kg,right_arm_load_kg,leg_load_kg,hip_load_kg,chest_load_kg
0,aaron-brogden-9y4MaTz2Js0-unsplash.jpg,6.05,6.64,83.88,18.98,12.87
1,pexels-shvetsa-4587381 (2).jpg,12.76,5.89,73.61,25.29,18.48
2,aaron-brogden-9y4MaTz2Js0-unsplash (1).jpg,6.05,6.64,83.88,18.98,12.87
3,anastase-maragos-iUzgePOoGko-unsplash (1).jpg,6.42,5.67,30.05,19.25,13.11
4,john-fornander-TAZoUmDqzXk-unsplash.jpg,7.58,6.22,36.63,18.86,12.76


✅ Load estimations saved to /content/load_estimations.csv
✅ Load estimations also saved to /content/drive/MyDrive/SEE_Assessment/load_estimations.csv
